In [1]:
from dotenv import load_dotenv
import os

import pandas as pd

In [2]:
# .env 파일 활성화
load_dotenv()

SERVICE_KEY = os.getenv('SERVICE_KEY')
PUBLIC_SERVICE_KEY = os.getenv('PUBLIC_SERVICE_KEY')

In [3]:
print(SERVICE_KEY, PUBLIC_SERVICE_KEY)

test test public


### Test Data
- 무신사 데이터 사용

In [4]:
recent_goods = {'goodsnm': '시티보이 빅 오버핏 빅사이즈 옥스포드 셔츠 4 COLOR',
                'goodsprc': 45900,
                'brand': '언탭트 스튜디오',
                'gender': '남',
                'season': '2024 FW'}
recommend_resuts = [
    {
        'goodsnm': '시티보이 빅 오버핏 그라데이션 타탄 체크 셔츠 딥블루',
        'goodsprc': 49957,
        'brand': '언탭트 스튜디오',
        'gender': '남',
        'season': '2024 FW'
    },
    {
        'goodsnm': '[SET]시티보이 빅오버핏 워시드 데님 셔츠 4Color',
        'goodsprc': 85336,
        'brand': '언탭트 스튜디오',
        'gender': '남',
        'season': '2025 ALL'
    },
    {
        'goodsnm': 'YKK 이지 세미와이드 슬랙스 3Color',
        'goodsprc': 46326,
        'brand': '언탭트 스튜디오',
        'gender': '남',
        'season': '2025 ALL'
    },
    {
        'goodsnm': 'KEYTH 핸드 크림 앳 허 아파트먼트 [GIFT 증정]',
        'goodsprc': 35000,
        'brand': '키스',
        'gender': '여'
    },
    {
        'goodsnm': '여성 클래식 데일리 백팩_WO1WCAA401',
        'goodsprc': 79900,
        'brand': '웨스트우드',
        'gender': '여'
    }
]

### gemini 모델 테스트

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel

## Gemini
# GCP 프로젝트 ID
project_id = os.getenv('project_id')
location = os.getenv('location')

# Vertex AI multimodal model initialize
vertexai.init(project=project_id, location=location)
model_gemini = GenerativeModel("gemini-1.5-flash-002")

In [6]:
prompt = f"""
목적: 추천결과를 평가하는 역할
데이터 형태:  
  - recent_goods: 최근 본 상품
  - result: 검색 추천 결과

평가 순서
1. 최근 본 상품과 추천 결과의 연관성 분류
  - 연관성은 0과 10 사이의 값으로 구분한다.
  - 높을수록 연관성이 높다.
2. 연관성 분류 이유 설명

형식은 json 포맷으로 해줘.

recent_goods: {recent_goods},

result: {recommend_resuts}
"""

In [7]:
responses = model_gemini.generate_content(
    prompt,
    generation_config={
        "temperature": 0.2,
        "top_p": 0.9,
    }
)

In [8]:
print(responses.text)

```json
{
  "evaluation": {
    "recent_goods": {
      "goodsnm": "시티보이 빅 오버핏 빅사이즈 옥스포드 셔츠 4 COLOR",
      "goodsprc": 45900,
      "brand": "언탭트 스튜디오",
      "gender": "남",
      "season": "2024 FW"
    },
    "results": [
      {
        "goodsnm": "시티보이 빅 오버핏 그라데이션 타탄 체크 셔츠 딥블루",
        "goodsprc": 49957,
        "brand": "언탭트 스튜디오",
        "gender": "남",
        "season": "2024 FW",
        "relevance": 9,
        "reason": "브랜드, 성별, 시즌이 동일하고, 상품명에 '시티보이 빅 오버핏 셔츠'라는 공통 키워드가 포함되어 있습니다.  디자인이 다르지만 같은 스타일의 셔츠이므로 높은 연관성을 가집니다."
      },
      {
        "goodsnm": "[SET]시티보이 빅오버핏 워시드 데님 셔츠 4Color",
        "goodsprc": 85336,
        "brand": "언탭트 스튜디오",
        "gender": "남",
        "season": "2025 ALL",
        "relevance": 7,
        "reason": "브랜드와 성별이 동일하고, 상품명에 '시티보이 빅오버핏 셔츠'라는 공통 키워드가 포함되어 있습니다. 시즌은 다르지만,  같은 스타일의 셔츠이므로 높은 연관성을 가집니다. 세트 상품이라는 점은 약간의 차이를 만듭니다."
      },
      {
        "goodsnm": "YKK 이지 세미와이드 슬랙스 3Color",
        "goodsprc": 46326,
        "brand": "언탭트 스튜디오",